## Casus WebTraffic

The case Webtraffic is based on following resources:
- <a href="https://www.packtpub.com/big-data-and-business-intelligence/building-machine-learning-systems-python-second-edition" target="_blank">Building Machine Learning with Python (2e)</a>
- <a href="https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-pyand " target="_blank">scikit-learn: Linear Regression Example</a>

# An application of machine learning

Let's get our hands dirty and take a look at our hypothetical web start-up, MLaaS, which sells the service of providing machine learning algorithms via HTTP. 

With increasing success of our company, the demand for better infrastructure increases to serve all incoming web requests successfully. We don't want to allocate too many resources as that would be too costly. On the other side, we will lose money, if we have not reserved enough resources to serve all incoming requests. 

Now, the question is, **when will we hit the limit of our current infrastructure, which we estimated to be at 100,000 requests per hour**. We would like to know in advance when we have to request additional servers in the cloud to serve all the incoming requests successfully without paying for unused ones.

Lets enter the data science pipeline.

In [ ]:
# display plots in Notebook cell
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

## Reading in the data 
File `web_traffic.tsv` contains web statistics for the last month and aggregated in the file. 

The data is stored as the number of hits per hour. Each line contains the hour consecutively and the number of web hits in that hour

**Activity**<br>
Open the file `web_traffic.tsv` with a text-editor, and look at it briefly.

In [ ]:
import scipy as sp
path = './datasets/'
data = sp.genfromtxt(path + "web_traffic.tsv", delimiter="\t")
# .tsv because it contains tab-separated values
#help(sp.genfromtxt

In [ ]:
# display some data
print (data[:10])

# display the dimensions of the data...
print('\n', data.shape)

type(data)

As you can see, we have 743 data points with 2 dimensions.
One caveat is still that we have some values in y that contain invalid values, <code>nan</code>.

## Preprocessing and cleaning the data

Model linear regression in two dimensions, so we separate the dimensions into two vectors, each of size 743. The first vector, X, will contain the hours, and the other, y, will contain the Web hits in that particular hour. 

This splitting is done using the special index notation of SciPy, by which we can choose the columns individually.

There are many more ways in which data can be selected from a SciPy array. 
Check out http://www.scipy.org/Tentative_NumPy_Tutorial for more details on indexing, slicing, and iterating.

In [ ]:
# Use only one feature
X = data[:, np.newaxis, 0]  # hours
# type(X)
print('Dimension feature X: {}'.format(X.shape))

In [ ]:
# target
y = data[:, np.newaxis, 1]  # traffic
# # type(y)
print('Dimension target y: {}'.format(y.shape))

y contains invalid values, <code>nan</code>. The question is what we can do with them?
Let's check how many hours contain invalid data, by running the following code:

In [ ]:
# cleanup the data - how many NaN?
import scipy as sp
sp.sum(sp.isnan(y))

### What to do with the invalid data?
As you can see, we are missing only 8 out of 743 entries, so we can afford to remove them.

We can index a SciPy array with another array. <b>sp.isnan(y)</b> returns an array of Booleans indicating whether an entry is a number or not. Using <b>~</b>, we logically negate that array so that we choose only those elements from x and y where y contains valid numbers.

In [ ]:
# cleanup the data - remove NaN?
X = X[~sp.isnan(y)]
y = y[~sp.isnan(y)]

# check there are no invalid numbers 'nan' anymore
print('X contains {} NaN:'.format(sp.sum(sp.isnan(X))))
print('y contains {} NaN:'.format(sp.sum(sp.isnan(y))))

### Exploring data visually

To get the first impression of our data, let's plot the data in a scatter plot using matplotlib. Matplotlib contains the pyplot package.

In [ ]:
# display plots in Notebook cell
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# plot the (X,y) points with dots of size 10
plt.scatter(X, y, s=10)

plt.title("Web traffic over the last month")
plt.xlabel("Time")
plt.ylabel("Hits/hour")
plt.xticks([w*7*24 for w in range(10)],
               ['week %i' % w for w in range(10)])
plt.autoscale(tight=True)
# draw a slightly opaque, dashed grid
plt.grid(True, linestyle='-', color='0.75')

plt.show()

In the resulting chart, we can see that while in the first weeks the traffic stayed more or less the same, the last week shows a steep increase.

## Linear regression model and learning algorithm
Now that we have a first impression of the data, we return to the initial question: **How long will our server handle the incoming web traffic?**

To answer this we have to do the following:

1. Find the real model behind the noisy data points (**training**).
2. Following this, use the model to extrapolate into the future to find the point in time where our infrastructure has to be extended (**prediction**).

### Build our first model

When we talk about models, you can think of them as simplified theoretical approximations of complex reality. As such there is always some inferiority involved, also called the **approximation error**. 
It is said to be a discrepancy between approximated value and exact value.

In [ ]:
# Split the data into training/testing sets
X_train = X[:-20]
X_test = X[-20:]
# type(X_train)

In [ ]:
# Split the targets into training/testing sets
y_train = y[:-20]
y_test = y[-20:]
# type(y_train)

In [ ]:
from sklearn.model_selection import train_test_split

"""
# splitting arrays manually is very error-prune!
X_train = X[:-20] # X[:-400
X_test = X[-20:]  # X[-400:]
y_train = y[:-20] # y[:-400]
y_test = y[-20:]  # y[-400:]
"""

# a better split in training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

In [ ]:
#Reshaping the array's for scikit-learn, since we only have a single feature
X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

In [ ]:
# Train the model using the training sets
regr.fit(X_train, y_train)

In [ ]:
#Reshaping the array's for scikit-learn, since we only have a single feature
X_test = X_test.reshape(-1, 1)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [ ]:
# The coefficient parameter (slope)
print('Slope: ', regr.coef_)
# The intercept parameter
print('Intercept: ', regr.intercept_)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

In [ ]:
accuracy = regr.score(X_test, y_test)
print("Prediction Accuracy: {:.2f}%".format(accuracy * 100))

### Plot the data and the prediction line

The prediction line is a simpel linear function with slope and intercept: `predict()` function.

Slope and intercept are provided by the regression model `regr`.

In [ ]:
slope = regr.coef_[0][0]
intercept = regr.intercept_[0]

def predict(x):
    return slope * x + intercept

fitline = predict(X)  # X is feature!

plt.title("Web traffic over the last month")
plt.xlabel("Time")
plt.ylabel("Hits/hour")
plt.xticks([w*7*24 for w in range(10)],
               ['week %i' % w for w in range(10)])
plt.autoscale(tight=True)
# draw a slightly opaque, dashed grid
plt.grid(True, linestyle='-', color='0.75')

# dataset
plt.scatter(X, y, s=10)
plt.scatter(X_test, y_test, s=20)

# prediction line
plt.plot(X, fitline, color='red', lineWidth=4)

plt.show()

#### Alternative prediction line

The prediction line is a polynomial of degree 1, slope and intercept is provided by the regression model `regr`, and the polynomial function is generated with help of SciPy's `polyd()` method.

In [ ]:
# prepare for plotting:

# fm represents slope and intercept for a polynomial of degree 1 (f1).
fm = np.array([regr.coef_[0][0], regr.intercept_[0]])
print("Model parameters: {}".format(fm)) # slope, intercept

# generate polynomial function
f1 = sp.poly1d(fm)  

In [ ]:
# plot the (test/data) points with dots of size 10

plt.title("Web traffic over the last month")
plt.xlabel("Time")
plt.ylabel("Hits/hour")
plt.xticks([w*7*24 for w in range(10)],
               ['week %i' % w for w in range(10)])
plt.autoscale(tight=True)
# draw a slightly opaque, dashed grid
plt.grid(True, linestyle='-', color='0.75')

# draw dataset and prediction line
plt.scatter(X, y, s=10)
plt.scatter(X_test, y_test, s=20)

# polynomial function f1()
#plt.plot(X_test, f1(X_test), linewidth=4, color='red')
plt.plot(X, f1(X), linewidth=4, color='red')

# generate X-values for f1()
# fx = sp.linspace(0, X[-1], 1000) 
# plt.plot(fx, f1(fx), linewidth=4, color='blue')

plt.legend(["d=%i" % f1.order], loc="upper left")
plt.show()

## Answering our initial question

Finally we have arrived at a model which we think represents the underlying process; it is now a simple task of finding out when our infrastructure will reach 100,000 requests per hour. We have to calculate when our model function reaches the value 100,000.

SciPy's `optimize` module has the function `fsolve` that achieves this, when providing an initial starting position with parameter `x0`.
* Having a polynomial of degree 1, we could simply compute the inverse of the function and calculate its value at 100,000. Of course, we would like to have an approach that is applicable to any model function easily.

* This can be done by subtracting 100,000 from the polynomial, which results in another polynomial, and finding its root. 


As every entry in our input data file corresponds to one hour, and we have 743 of them, we set the starting position to some value after that. Let `f1` be the polynomial of degree 1.

In [ ]:
# print the polynomial function

print("f1(x)= {}\n".format(f1))
print("f1(x)-100,000= {}\n".format((f1-100000)))

# calculate the week of the 100000 hits/hour ...

from scipy.optimize import fsolve
reached_max = fsolve(f1-100000, x0=800)/(7*24)
print("100,000 hits/hour expected at week {0:4.0f}".format(reached_max[0]))

----

# Polynomial regression
using higher degree polynomials

## Optional reading.

Using SciPy's `polyfit()` method to provide higher degree polynomials than the linear model: see Jupyter notebook `casus_webtraffic_scipy.ipynb`.

**Result**: 100,000 hits/hour expected at week ±9.9


# Summary

You just learned two important things, of which the most important one is that as a typical machine learning operator, you will spend most of your time in understanding and refining the data.

And we hope that this example helped you to  start switching your mental focus from algorithms to data. Then you learned how important it is to have the correct experiment setup and that it is vital to not mix up training and testing.